In [21]:
!pip install pyarrowfs-adlgen2
!pip install azure-identity
!pip install azure-storage-file-datalake
!pip install duckdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.1/264.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.9/412.9 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.0 MB/s eta 0:00:00


In [22]:
!pwd

/content


In [23]:
# import tensorflow_decision_forests as tfdf
import ydf
import numpy as np
import pandas as pd
import tensorflow as tf
import tf_keras
import os
import duckdb

import pyarrow as pa
import pyarrow.parquet as pq

from dotenv import load_dotenv
from pathlib import Path

In [24]:
# Retrieve credentials from environment variables
# this is strictly used only in development
# load env variables
env_dir = Path('./').resolve()
load_dotenv(os.path.join(env_dir, '.env'))

True

In [25]:
MOTHERDUCK_TOKEN = os.environ.get("MOTHERDUCK_TOKEN")

In [26]:
conn = duckdb.connect("md:", config={"motherduck_token": MOTHERDUCK_TOKEN})

In [27]:
conn.sql("SELECT CURRENT_DATABASE()")

┌────────────────────┐
│ current_database() │
│      varchar       │
├────────────────────┤
│ my_db              │
└────────────────────┘

In [28]:
conn.sql("USE signal_gender_predictor_db")

In [29]:
conn.sql("SELECT CURRENT_DATABASE()")

┌────────────────────────────┐
│     current_database()     │
│          varchar           │
├────────────────────────────┤
│ signal_gender_predictor_db │
└────────────────────────────┘

convert the

In [30]:
# returns the pyarrow table as a dictionary e.g.
# {
#     'f1': [...],
#     'f2': [...],
#     ...
#     'fn': [...],
#     'label': [...]
# }
train_data_sc_sm_red_dict = conn.sql("SELECT * FROM train_data_sc_sm_red").fetchnumpy()
val_data_sc_sm_red_dict = conn.sql("SELECT * FROM val_data_sc_sm_red").fetchnumpy()
test_data_sc_sm_red_dict = conn.sql("SELECT * FROM test_data_sc_sm_red").fetchnumpy()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Cast the label of the numpy dictionary to an int instead of float

In [31]:
train_data_sc_sm_red_dict["label"] = train_data_sc_sm_red_dict["label"].astype(int)
val_data_sc_sm_red_dict["label"] = val_data_sc_sm_red_dict["label"].astype(int)
test_data_sc_sm_red_dict["label"] = test_data_sc_sm_red_dict["label"].astype(int)

In [32]:
len(train_data_sc_sm_red_dict)

61

In [33]:
a = list(train_data_sc_sm_red_dict.keys())

In [34]:
b = ["freq_kurt_imp", "freq_skew_imp", "freq_entropy_imp", "freq_mean_imp", "freq_median_imp", "freq_mode_imp", "freq_min_imp", "freq_max_imp", "freq_var_imp", "freq_stddev_imp", "freq_first_quart_imp", "freq_third_quart_imp", "freq_range_imp", "freq_inter_quart_range_imp", "zcr_imp", "poly_feat_1_imp", "poly_feat_2_imp", "spec_cent_imp", "spec_bw_imp", "spec_flat_imp", "spec_roll_imp", "mel_spec_mean_imp", "mel_spec_median_imp", "mel_spec_mode_imp", "mel_spec_min_imp", "mel_spec_max_imp", "mel_spec_range_imp", "mel_spec_var_imp", "mel_spec_std_imp", "mel_spec_first_quart_imp", "mel_spec_entropy_imp", "mel_spec_db_mean_imp", "mel_spec_db_median_imp", "mel_spec_db_mode_imp", "mel_spec_db_min_imp", "mel_spec_db_max_imp", "mel_spec_db_range_imp", "mel_spec_db_first_quart_imp", "mel_spec_db_third_quart_imp", "mel_spec_db_entropy_imp", "mel_spec_db_kurt_imp", "mel_spec_db_skew_imp", "mfcc_mean_imp", "mfcc_median_imp", "mfcc_mode_imp", "mfcc_min_imp", "mfcc_first_quart_imp", "mfcc_third_quart_imp", "mfcc_inter_quart_range_imp", "mfcc_kurt_imp", "mfcc_skew_imp", "spec_cont_mean_imp", "spec_cont_median_imp", "spec_cont_max_imp", "spec_cont_var_imp", "spec_cont_std_imp", "spec_cont_first_quart_imp", "spec_cont_third_quart_imp", "spec_cont_inter_quart_range_imp", "spec_cont_entropy_imp"]

In [35]:
set(a) - set(b)

{'label'}

In [36]:
ydf.verbose()
model = ydf.GradientBoostedTreesLearner(
    label="label",
    include_all_columns=True,
    task=ydf.Task.CLASSIFICATION,
).train(train_data_sc_sm_red_dict, val_data_sc_sm_red_dict)

max_vocab_count = -1 for column label, the dictionary will not be pruned by size.


Train model on 367776 training examples and 80458 validation examples


"goss_alpha" set but "sampling_method" not equal to "GOSS".
"goss_beta" set but "sampling_method" not equal to "GOSS".
"selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
Data spec:
Number of records: 367776
Number of columns: 61

Number of columns by type:
	NUMERICAL: 60 (98.3607%)
	CATEGORICAL: 1 (1.63934%)

Columns:

NUMERICAL: 60 (98.3607%)
	1: "freq_kurt_imp" NUMERICAL mean:0.0499645 min:-2.09779 max:11.1224 sd:0.922331 dtype:DTYPE_FLOAT64
	2: "freq_skew_imp" NUMERICAL mean:-0.0387909 min:-5.36401 max:4.50162 sd:0.900304 dtype:DTYPE_FLOAT64
	3: "freq_entropy_imp" NUMERICAL mean:0.383501 min:-2.38638 max:2.59962 sd:1.18057 dtype:DTYPE_FLOAT64
	4: "freq_mean_imp" NUMERICAL mean:0.0086529 min:-16.8118 max:16.8136 sd:0.820291 dtype:DTYPE_FLOAT64
	5: "freq_median_imp" NUMERICAL mean:0.0126425 min:-17.2178 max:17.2152 sd:0.817152 dtype:DTYPE_FLOAT64
	6: "freq_mode_imp" NUMERICAL mean:0.00881476 min:-10.5104 max:10.5074 sd:0.843662 dtype:DTYPE_FLOAT64
	7: 

ValueError: INVALID_ARGUMENT: Operation interrupted by user

In [37]:
  from google.colab import drive

  drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
MODEL_SAVE_PATH = "./drive/MyDrive/Colab Notebooks/signal-gender-predictor/"


In [ ]:
model.save(MODEL_SAVE_PATH)

In [40]:
signal_gender_predictor = ydf.load_model(MODEL_SAVE_PATH)
signal_gender_predictor

Loading model from ./drive/MyDrive/Colab Notebooks/signal-gender-predictor/


In [41]:
signal_gender_predictor.plot_tree()

In [42]:
evaluation = signal_gender_predictor.evaluate(test_data_sc_sm_red_dict)

Engine "GradientBoostedTreesQuickScorerExtended" built


In [43]:
evaluation

Label \ Pred,0,1
0,38043,3197
1,5598,35642


In [50]:
test = test_data_sc_sm_red_dict.copy()
test_label = test.pop("label")

In [51]:
test_label

array([0, 0, 0, ..., 1, 1, 1])

In [52]:
np.unique(test_label, return_counts=True)

(array([0, 1]), array([41240, 41240]))

In [53]:
preds = signal_gender_predictor.predict(test)
decoded = {"predictions": ["female" if pred else "male" for pred in (preds >= 0.5).tolist()]}
np.unique(decoded["predictions"], return_counts=True)

(array(['female', 'male'], dtype='<U6'), array([38839, 43641]))

In [ ]:
import pyarrow as pa

# Create a PyArrow Table
data = {
    'name': ['Alice', 'Bob', 'Charlie'],
    'age': [30, 24, 35],
    'city': ['New York', 'Los Angeles', 'Chicago']
}
table = pa.Table.from_pydict(data)

# Convert the Table to a Python dictionary
python_dict = table.to_pydict()

print(python_dict)